# 0. Configuration

In [1]:
MODEL_NAME = "google/flan-t5-large"

BASE_LOCATION = "/content/drive/My Drive/"
TRAIN_LOCATION = BASE_LOCATION + "train_top_20.csv"
VAL_LOCATION = BASE_LOCATION + "val_top_20.csv"

LEARNING_RATE = 3e-5
BATCH_SIZE = 4
NUM_EPOCHS = 1

# 1. Install dependencies

In [2]:
!pip install -q --upgrade transformers datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-many

# 2. Load model

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# 3. Load dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
from datasets import Dataset
from ast import literal_eval

train_dataset = pd.read_csv(TRAIN_LOCATION, index_col=0)
train_dataset['evidences'] = train_dataset['evidences'].apply(literal_eval)
train_dataset['scores'] = train_dataset['scores'].apply(literal_eval)
train_dataset = Dataset.from_pandas(train_dataset)

val_dataset = pd.read_csv(VAL_LOCATION, index_col=0)
val_dataset['evidences'] = val_dataset['evidences'].apply(literal_eval)
val_dataset['scores'] = val_dataset['scores'].apply(literal_eval)
val_dataset = Dataset.from_pandas(val_dataset)

# 4. Preprocess data

In [41]:
def generate_prompt(claim, evidences):
    prompt_begin = (
        f"You are a fact-checker. Your task is to classify the claim based on the following evidence.\n\n"
        f"Claim: {claim}\n\n"
        f"Evidences:\n"
    )
    prompt_evidences = f""
    prompt_end = f"\n\nLabel (TRUE, FALSE, CONFLICTING):"

    for k in range(0, min(5, len(evidences))):
        new_evidence = f"\n{k + 1}. {evidences[k][0]} (score: {round(evidences[k][1], 2)})"
        tokens = tokenizer(prompt_begin + prompt_evidences + new_evidence + prompt_end, max_length=512, truncation=True, return_tensors="pt")
        if tokens['input_ids'].shape[1] < tokenizer.model_max_length:
            prompt_evidences += new_evidence
        else:
            break
    return prompt_begin + prompt_evidences + prompt_end

def preprocess_function(examples):
    inputs = [generate_prompt(examples['claim'][i], list(zip(examples['evidences'][i], examples['scores'][i]))) for i in range(len(examples["claim"]))]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["label"], max_length=16, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [43]:
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/9935 [00:00<?, ? examples/s]

Map:   0%|          | 0/3084 [00:00<?, ? examples/s]

# 4. Prepare training

In [49]:
from datasets import load_metric
import numpy as np

metric = load_metric("f1")

def to_number(label):
  match label:
    case "TRUE":
      return 0
    case "FALSE":
      return 1
    case "CONFLICTING":
      return 2
    case _:
      return 3

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [to_number(pred) for pred in decoded_preds]
    decoded_labels = [to_number(label) for label in decoded_labels]

    f1_weighted = round(metric.compute(predictions=decoded_preds, references=decoded_labels, average="weighted")['f1'], 4)
    f1_macro = round(metric.compute(predictions=decoded_preds, references=decoded_labels, average="macro")['f1'], 4)
    return {"f1_weighted": f1_weighted, "f1_macro": f1_macro}

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [50]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
import torch

# Step 4: Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    weight_decay=0.01,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Step 5: Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 5. Train

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/content/drive/My Drive/big_boi")

# 6. Eval

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/big_boi')

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
